<a href="https://colab.research.google.com/github/LucasFerrSil/GrafosDoBitcoin/blob/main/CR_CRIA%C3%87%C3%83O_DAS_TABELAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Para os grafos representando o ano inteiro, fazer função distribuição discreta para o grau de cada input e de cada output
#dps criar grafos com um número consideravel de arestas e nós e buscar a lei de potencia

def distribuicao(lista):
  minl = min(lista)
  maxl = max(lista)
  n = int(len(lista)**(1/2))
  inter = ((maxl-minl)/n)
  x = []
  for i in range(n):
    v = 0
    for j in range(len(lista)):
      if(minl+inter*i <= lista[j] and lista[j] <= (minl+inter*(i+1))):
        v += 1
    x.append(v)
  soma = sum(x)
  f = [x[i]/soma for i in range(len(x))]
  intervalo = [[minl+inter*i,minl+inter*(i+1)] for i in range(n)]
  return intervalo,f

In [ ]:
import requests
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hash = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/HashBlocks.xlsx')
hashBlocks = list(hash['Hash'][0:10])
display(hashBlocks)

['000000000000000000065afb7a27cd94412ef1ebbaf91f0f9d8bb891ff165d90',
 '00000000000000000003cf7e32b1e6e614f48f9e31adafd58031156dc748a6b6',
 '00000000000000000001695fa2f44e60467c36ddfa67e849f44f765b96c98307',
 '0000000000000000000438733a43c41f0111819de5e1744e9855649b3d824165',
 '00000000000000000004f312afaed7e79d6e0e2e17eaf891c2d5943255bbb0ae',
 '00000000000000000001ebdf5823090f8f6f46db59549f027b452b90392edaba',
 '000000000000000000047d6e72dd6e0a0a1ac4e4101a0d89b625480162ba9363',
 '0000000000000000000247ebc21bcc220f73a39848779c10866b3ce78e42c97c',
 '00000000000000000000b93369ca030defdc146dbbc93e1c9741e91d0957d7e4',
 '00000000000000000002f2ca4135e9d1e836b234f344e63bc916b07ba0783025']

In [ ]:
api_url = ['https://blockchain.info/rawblock/' + str(hashBlocks[i]) for i in range(len(hashBlocks))]
response = [requests.get(api_url[i]).json() for i in range(len(api_url))]
tx_data = [response[i]['tx'] for i in range(len(response))]

In [ ]:
df = pd.DataFrame(tx_data)
df.to_csv('/content/drive/MyDrive/Nova pasta/tranxinfo2023.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Nova pasta/tranxinfo2023.csv')
tx_data2 = df.values
tx_data3 = []
for i in range(len(tx_data2)):
  vetor = [(transacao) for transacao in tx_data2[i] if transacao != None]
  tx_data3.append(vetor)

In [ ]:
#Inputs e outputs atraves da Blockchain.com

addresses_input = []
addresses_output = []
addresses_input_t = []
addresses_output_t = []
for i in range(len(tx_data)):
  inputs = []
  outputs = []
  input = []
  output = []
  for j in range(len(tx_data[i])):
    for k in range(len(tx_data[i][j]['inputs'])):
      if(j != 0):
        inputs.append([tx_data[i][j]['inputs'][k]['prev_out']['addr'],tx_data[i][j]['inputs'][k]['prev_out']['value'],tx_data[i][j]['hash']])
        input.append([tx_data[i][j]['inputs'][k]['prev_out']['addr'],tx_data[i][j]['inputs'][k]['prev_out']['value']])
      else:
        inputs.append(['Inicial',0,tx_data[i][j]['hash']])
        input.append(['Inicial',0])
    for w in range(len(tx_data[i][j]['out'])):
      if(tx_data[i][j]['out'][w]['spent'] == True):
        outputs.append([tx_data[i][j]['out'][w]['addr'],tx_data[i][j]['out'][w]['value'],tx_data[i][j]['hash']])
        output.append([tx_data[i][j]['out'][w]['addr'],tx_data[i][j]['out'][w]['value']])
  addresses_input_t.append(inputs)
  addresses_output_t.append(outputs)
  addresses_input.append(input)
  addresses_output.append(output)

In [ ]:
inputs_tabela = pd.DataFrame(addresses_input_t)
outputs_tabela = pd.DataFrame(addresses_output_t)
inputs_tabela.to_csv('/content/drive/MyDrive/Nova pasta/inputTx2023.csv',index=False)
outputs_tabela.to_csv('/content/drive/MyDrive/Nova pasta/outputTx2023.csv',index=False)

input = pd.DataFrame(addresses_input)
output = pd.DataFrame(addresses_output)
input.to_csv('/content/drive/MyDrive/Nova pasta/inputs2023.csv',index=False)
output.to_csv('/content/drive/MyDrive/Nova pasta/outputs2023.csv',index=False)

In [ ]:
def tabela(vi,vo):
  valores_output = [vo[i][1] for i in range(len(vo))]
  valores_input = [vi[i][1] for i in range(len(vi))]
  soma_input = sum(valores_input)
  lista = []
  for i in range(len(vi)):
    termo = []
    for j in range(len(vo)):
      k = valores_output[j]/soma_input
      lista.append(k*valores_input[i])
  edges = []
  for i in range(len(vi)):
    for j in range(len(vo)):
      edges.append([vi[i][0],vo[j][0]])
  for i in range(len(lista)):
    edges[i].append(lista[i])
  return edges

In [ ]:
e = []
for i in range(len(addresses_input)):
    e.append(tabela(addresses_input[i],addresses_output[i]))

In [ ]:
input = [df['Input'][i] for i in range(len(df['Input']))]
output = [[df['Output'][i],df['Valor'][i]] for i in range(len(df['Input']))]
copyinput = input
copyoutput = output
lista = []
i = 0
while i != len(copyinput):
  indices = []
  termo = []
  input_analisado = copyinput[i]
  termo.append(copyinput[i])
  termo.append(copyoutput[i])
  for k in range(len(copyinput)):
    if(k != i and copyinput[k]==copyinput[i]):
      indices.append(k)
  for indice in indices:
    termo.append(output[indice])
    copyinput.remove(input_analisado)
  lista.append(termo)
  i += 1

In [ ]:
edges = []
for i in range(len(lista)):
  for j in range(1,len(lista[i])):
    termo = (lista[i][0],lista[i][j][0],lista[i][j][1])
    edges.append(termo)

('1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', '1J4xzYnppUBR46tewFxMDWCsQkMJG3hHaU', 260000.0)
('1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', 'bc1qwj0lzfqn0ydhr0dpec5pur00tmkcyh3lsg7d4c', 472172.0)
('1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', '1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', 502815.0)
('1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', 'bc1qakeyagrj395yfvf3vayxwwmzps699mq9uh9nn2', 451872.0)
('1Q1DJtJYTXc1QBQWYHYqZhPPyMykzpfSAq', '127nE9ME5jLMrjHp299nC5KzBFF8ExqkcP', 21843542.0)
